In [2]:
!pip install transformers torch datasets sentence-transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00


In [2]:
# import os
# import PyPDF2

# def extract_text_from_pdf(pdf_path):
#     """Extract text from a PDF file."""
#     text = ""
#     try:
#         with open(pdf_path, "rb") as file:
#             reader = PyPDF2.PdfReader(file)
#             for page in reader.pages:
#                 text += page.extract_text() + "\n"
#     except Exception as e:
#         print(f"Error extracting {pdf_path}: {e}")
#     return text

# def parse_resume_dataset(root_dir):
#     """Parse through the dataset directory and extract text from PDFs."""
#     job_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
    
#     resume_data = {}
    
#     for job in job_folders:
#         job_path = os.path.join(root_dir, job)
#         resume_data[job] = []
        
#         for filename in os.listdir(job_path):
#             if filename.lower().endswith(".pdf"):
#                 pdf_path = os.path.join(job_path, filename)
#                 text = extract_text_from_pdf(pdf_path)
#                 resume_data[job].append({"filename": filename, "content": text})
    
#     return resume_data

# # Define dataset root directory
# dataset_path = "/kaggle/input/resume-dataset/data/data"
# resumes = parse_resume_dataset(dataset_path)

# # Print sample output
# for job, files in resumes.items():
#     print(f"Job Category: {job}")
#     for resume in files[:2]:  # Show only first two resumes per job
#         print(f"Filename: {resume['filename']}")
#         print(f"Content Preview: {resume['content'][:500]}\n")


In [3]:
# import os
# import PyPDF2
# import json

# def extract_text_from_pdf(pdf_path):
#     """Extract text from a PDF file."""
#     text = ""
#     try:
#         with open(pdf_path, "rb") as file:
#             reader = PyPDF2.PdfReader(file)
#             for page in reader.pages:
#                 text += page.extract_text() + "\n"
#     except Exception as e:
#         print(f"Error extracting {pdf_path}: {e}")
#     return text

# def load_checkpoint(checkpoint_path):
#     """Load checkpoint if exists."""
#     if os.path.exists(checkpoint_path):
#         with open(checkpoint_path, "r") as file:
#             return json.load(file)
#     return {}

# def save_checkpoint(checkpoint_path, data):
#     """Save checkpoint to file."""
#     with open(checkpoint_path, "w") as file:
#         json.dump(data, file)

# def parse_resume_dataset(root_dir, output_dir, checkpoint_path):
#     """Parse through the dataset directory and extract text from PDFs."""
#     job_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
#     resume_data = load_checkpoint(checkpoint_path)
    
#     for job in job_folders:
#         job_path = os.path.join(root_dir, job)
#         if job not in resume_data:
#             resume_data[job] = []
        
#         processed_files = {r["filename"] for r in resume_data[job]}
        
#         for filename in os.listdir(job_path):
#             if filename.lower().endswith(".pdf") and filename not in processed_files:
#                 pdf_path = os.path.join(job_path, filename)
#                 text = extract_text_from_pdf(pdf_path)
#                 resume_data[job].append({"filename": filename, "content": text})
                
#                 # Save checkpoint after each file
#                 save_checkpoint(checkpoint_path, resume_data)
    
#     output_file = os.path.join(output_dir, "resume_texts.json")
#     with open(output_file, "w") as f:
#         json.dump(resume_data, f, indent=4)
    
#     return resume_data

# # Define dataset root directory and output directory
# dataset_path = "/kaggle/input/resume-dataset/data/data"
# output_path = "/kaggle/working"
# checkpoint_file = os.path.join(output_path, "checkpoint.json")

# resumes = parse_resume_dataset(dataset_path, output_path, checkpoint_file)

# # Print sample output
# for job, files in resumes.items():
#     print(f"Job Category: {job}")
#     for resume in files[:2]:  # Show only first two resumes per job
#         print(f"Filename: {resume['filename']}")
#         print(f"Content Preview: {resume['content'][:500]}\n")


In [4]:
# import os
# import json
# import torch
# from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
# from datasets import Dataset

# # Load the JSON file containing resume texts
# def load_resume_data(json_path):
#     with open(json_path, "r") as f:
#         resume_data = json.load(f)
#     return resume_data

# # Preprocess the resume data for training
# def preprocess_data(resume_data):
#     inputs = []
#     labels = []
#     for job, resumes in resume_data.items():
#         for resume in resumes:
#             text = resume["content"]
#             # Tokenize the text
#             tokens = tokenizer(
#                 text,
#                 truncation=True,
#                 padding="max_length",
#                 max_length=512,  # Adjust based on your needs
#                 return_tensors="pt",
#             )
#             inputs.append(tokens)
            
#             # Create pseudo-labels (replace this with actual labels if available)
#             # For now, we use a dummy label (0) for all tokens
#             label = [0] * len(tokens["input_ids"][0])  # Dummy label
#             labels.append(label)
    
#     # Combine inputs and labels into a dataset
#     dataset = {
#         "input_ids": torch.cat([x["input_ids"] for x in inputs]),
#         "attention_mask": torch.cat([x["attention_mask"] for x in inputs]),
#         "labels": torch.tensor(labels),
#     }
#     return Dataset.from_dict(dataset)

# # Load pre-trained model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=8)  # Adjust num_labels as needed

# # Load the JSON file
# json_path = os.path.join(output_path, "resume_texts.json")
# resume_data = load_resume_data(json_path)

# # Preprocess the data
# dataset = preprocess_data(resume_data)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="no",  # Disable evaluation
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=2,
#     save_steps=500,
#     logging_dir="./logs",
#     logging_steps=10,
# )

# # Define Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
# )

# # Fine-tune the model
# trainer.train()

# # Save the fine-tuned model
# trainer.save_model("./fine-tuned-resume-model")
# tokenizer.save_pretrained("./fine-tuned-resume-model")

In [5]:
# Enhanced patterns with better context awareness and coverage
PATTERNS = {
    "name": [
        # Handles multi-part names, hyphens, and international characters
        r"(?i)\b(?:[A-Z][a-z]+(?:[-'\s][A-Z][a-z]+){1,3})\b",
        # Matches name headers (e.g., "Name:", "Full Name:")
        r"(?i)(?:name|full name|contact name):?\s*([A-Z][a-z]+(?: [A-Z][a-z]+){1,3})"
    ],
    
    "email": [
        # Improved email pattern with common resume formats
        r"(?i)(?:email|e-mail|contact):?\s*([a-z0-9_.+-]+@[a-z0-9-]+\.[a-z0-9-.]{2,})",
        r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b"
    ],
    
    "phone": [
        # Comprehensive phone number patterns with optional country codes
        r"(?i)(?:phone|mobile|tel):?\s*(\+?\d[\d\s-]{7,15})",
        r"\b\+\d{1,3}[- ]?\(?\d{3}\)?[- ]?\d{3}[- ]?\d{4}\b",
        r"\b\d{3}[- ]?\d{3}[- ]?\d{4}\b"
    ],
    
    "skill": [
        # Skills with context awareness (section headers and bullet points)
        r"(?i)(?:skills|technical skills|competencies):?[\s\S]*?([-\•].*?)(?=\n\s*\n)",
        # Programming languages with version numbers
        r"\b(?:python|java|c\+\+)(?:\s*\d+\.?\d*)?\b",
        # Cloud technologies with platforms
        r"\b(?:aws|azure|gcp)(?:\s*\(?(?:ec2|s3|lambda)\)?)?\b",
        # Frameworks with version ranges
        r"\b(?:react(?:\.js)?|angular|vue)(?:\s*\d+\.?\d*[- ]?\d*\.?\d*)?\b",
        # Certifications in skills section
        r"\b(?:certified|licensed)\s+(?:aws|scrum|pmp)\b"
    ],
    
    "experience": [
        # Job title with company and duration pattern
        r"(?i)(\b(?:senior|junior)?\s*(?:software engineer|data scientist)\b.*?\b(?:at|@)\b.*?\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b\s*\d{4}.*?–.*?(?:present|\d{4}))",
        # Duration patterns with multiple formats
        r"\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\s*\d{4}\s*(?:–|-|to)\s*(?:present|current|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\s*\d{4})\b",
        # Achievement-based experience points
        r"-\s*(?:led|developed|implemented|optimized)\s+.*?\b(?:using|with)\b.*?\b(?:python|machine learning|aws)\b"
    ],
    
    "certification": [
        # Certification with issuing authority and date
        r"(?i)(?:certifications|licenses):?[\s\S]*?-\s*(.*?)\s*(?:\(?(?:aws|microsoft|google)\)?)\s*(?:\(\d{4}\))?",
        # Specific certification patterns
        r"\b(?:AWS Certified Solutions Architect|Google Cloud Professional|PMP®)\b(?:\s*–?\s*\d{4})?",
        # Certification IDs
        r"\b(?:certification\s+ID:?\s*[A-Z0-9-]+)\b"
    ]
}



In [6]:
# import re
# import json
# import random
# import numpy as np
# from collections import defaultdict
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Input, Concatenate
# from tensorflow.keras.optimizers import Adam
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.base import BaseEstimator, TransformerMixin

# # Enhanced patterns with context awareness
# PATTERNS = {
#     "name": [
#         r"(?i)\b(?:[A-Z][a-z]+(?:[-'\s][A-Z][a-z]+){1,3})\b",
#         r"(?i)(?:name|full name|contact name):?\s*([A-Z][a-z]+(?: [A-Z][a-z]+){1,3})"
#     ],
#     "email": [
#         r"(?i)(?:email|e-mail|contact):?\s*([a-z0-9_.+-]+@[a-z0-9-]+\.[a-z0-9-.]{2,})",
#         r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b"
#     ],
#     "phone": [
#         r"(?i)(?:phone|mobile|tel):?\s*(\+?\d[\d\s-]{7,15})",
#         r"\b\+\d{1,3}[- ]?\(?\d{3}\)?[- ]?\d{3}[- ]?\d{4}\b"
#     ],
#     "skill": [
#         r"(?i)(?:skills|technical skills|competencies):?[\s\S]*?([-\•].*?)(?=\n\s*\n)",
#         r"\b(?:python|java|c\+\+)(?:\s*\d+\.?\d*)?\b",
#         r"\b(?:aws|azure|gcp)(?:\s*\(?(?:ec2|s3|lambda)\)?)?\b"
#     ],
#     "experience": [
#         r"(?i)(\b(?:senior|junior)?\s*(?:software engineer|data scientist)\b.*?\b(?:at|@)\b.*?\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b\s*\d{4}.*?–.*?(?:present|\d{4}))",
#         r"-\s*(?:led|developed|implemented|optimized)\s+.*?\b(?:using|with)\b.*?\b(?:python|machine learning|aws)\b"
#     ],
#     "certification": [
#         r"(?i)(?:certifications|licenses):?[\s\S]*?-\s*(.*?)\s*(?:\(?(?:aws|microsoft|google)\)?)\s*(?:\(\d{4}\))?",
#         r"\b(?:AWS Certified Solutions Architect|Google Cloud Professional|PMP®)\b(?:\s*–?\s*\d{4})?"
#     ]
# }

# ACTIONS = list(PATTERNS.keys()) + ["next"]

# REWARDS = {
#     "correct": 2.0,
#     "context_match": 1.5,
#     "partial": 0.5,
#     "incorrect": -1.0,
#     "neutral": -0.1,
#     "redundant": -0.7
# }

# class EnhancedVectorizer(BaseEstimator, TransformerMixin):
#     def __init__(self, max_features=100):
#         self.vectorizer = CountVectorizer(max_features=max_features)
        
#     def fit(self, X, y=None):
#         text_data = [x.split("[FEATURES_START]")[0].strip() for x in X]
#         self.vectorizer.fit(text_data)
#         return self
        
#     def transform(self, X):
#         text_data = [x.split("[FEATURES_START]")[0].strip() for x in X]
#         features = [json.loads(x.split("[FEATURES_START]")[1].split("[FEATURES_END]")[0]) for x in X]
#         text_vectors = self.vectorizer.transform(text_data).toarray()
#         feature_vectors = np.array([[f['has_date'], f['has_company'], f['has_title'], f['list_item']] for f in features])
#         return np.concatenate([text_vectors, feature_vectors], axis=1)

# class EnhancedResumeEnv:
#     def __init__(self, resume_text):
#         self.lines = [line.strip() for line in resume_text.split("\n") if line.strip()]
#         self.current_line = 0
#         self.extracted_data = defaultdict(list)
        
#     def get_state_representation(self, line):
#         features = {
#             'has_date': bool(re.search(r'\d{4}', line)),
#             'has_company': bool(re.search(r'\b(?:at|@)\b', line)),
#             'has_title': bool(re.search(r'\b(?:experienced|senior|junior)\b', line)),
#             'list_item': bool(re.search(r'^[-\•]', line))
#         }
#         return f"{line} [FEATURES_START]{json.dumps(features)}[FEATURES_END]"
    
#     def reset(self):
#         self.current_line = 0
#         self.extracted_data = defaultdict(list)
#         if self.lines:
#             return self.get_state_representation(self.lines[0])
#         return None
    
#     def step(self, action):
#         try:
#             if self.current_line >= len(self.lines):
#                 return None, 0, True, self.extracted_data
                
#             raw_line = self.lines[self.current_line]
#             processed_line = self.get_state_representation(raw_line)
#             reward = REWARDS["neutral"]
#             done = False
#             extracted = False
            
#             if action in PATTERNS:
#                 # Check for section headers
#                 if any(re.search(rf"(?i)^{key}:?", raw_line) for key in PATTERNS.keys()):
#                     reward = REWARDS["context_match"]
                    
#                 # Pattern matching
#                 for pattern in PATTERNS[action]:
#                     match = re.search(pattern, raw_line, re.IGNORECASE)
#                     if match:
#                         content = match.group(1) if len(match.groups()) > 0 else match.group()
#                         if content not in self.extracted_data[action]:
#                             self.extracted_data[action].append(content.strip())
#                             reward = REWARDS["correct"]
#                         else:
#                             reward = REWARDS["redundant"]
#                         extracted = True
#                         break
#                 if not extracted:
#                     reward = REWARDS["incorrect"]

#             self.current_line += 1
#             next_state = self.get_state_representation(self.lines[self.current_line]) if self.current_line < len(self.lines) else None
#             done = self.current_line >= len(self.lines)
            
#             return next_state, reward, done, self.extracted_data
#         except IndexError:
#             return None, 0, True, self.extracted_data

# class PolicyNetwork:
#     def __init__(self, state_size, action_size):
#         self.state_size = state_size
#         self.action_size = action_size
#         self.model = self._build_model()
#         self.optimizer = Adam(0.001)
        
#     def _build_model(self):
#         model = Sequential([
#             Dense(64, activation='relu', input_shape=(self.state_size,)),
#             Dense(64, activation='relu'),
#             Dense(self.action_size, activation='softmax')
#         ])
#         return model
    
#     def choose_action(self, state):
#         probs = self.model.predict(state, verbose=0)[0]
#         return np.random.choice(self.action_size, p=probs)
    
#     def train(self, states, actions, rewards):
#         with tf.GradientTape() as tape:
#             probs = self.model(states, training=True)
#             selected_probs = tf.reduce_sum(probs * tf.one_hot(actions, self.action_size), axis=1)
#             loss = -tf.reduce_mean(tf.math.log(selected_probs) * rewards)
#         grads = tape.gradient(loss, self.model.trainable_variables)
#         self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

# # Load resume data
# resume_data = {
#     "DESIGNER": [
#         {
#             "filename": "22506245.pdf",
#             "content": "PRE-PRESS GRAPHIC DESIGNER\nSummary\nCreative, hardworking designer seeking a full-time desktop job, educated as a graphic artist, past experience in business world as a desktop\npublisher laying out designs for printed mail and advertisements, in local government designing new websites with graphics for different agencies\nwithin the system, and later for the same government printing and reproduction center creating documents to be printed off a press or copiers.\nSkills\nAdobe InDesign, Photoshop, Illustrator, and Acrobat Professional\nStrongly familiar with Microsoft Word, Excel, PowerPoint, and Publisher / also QuarkXPress\nBasic knowledge of web development with Adobe Dreamweaver, HTML, WordPress\nAble to perform graphic design and administrative functions\nAble to work as a team player and independently\nExperienced using phone, fax, email, copiers and printers\nProvides excellent customer service (in-person, by phone, email, or interoffice mail)\nPrioritizes and calmly handles multiple projects and requests\nListens to directions, takes notes for later reference, follows procedures\nKnowledge of design setup on computer for jobs to be printed by outside vender or in-office copiers\nExperience\n01/2008\n \nto \nCurrent\nPre-Press Graphic Designer\n \nCompany Name\n \n\u00ef\u00bc\u200b \nCity\n \n, \nState\nCreate new designs for variety of items like manuals, newsletters, and posters.\nUse templates for updated documents like envelopes, letterheads, and business cards.\nProof jobs for initial and final customer approval.\nManufactures a high-quality PDF file digitally for proofing, photocopying and offset printing.\nPerforms file backup and organizes system for easy recovery.\nMaintains and monitors supply inventory and orders items when needed.\nOperates photocopying equipment, includes sending approved documents to printer.\nAssists in the bindery department, using the folder and manual paper cutter for small jobs.\nAlso can use bindery equipment, like the fastback and GBC binding of spines.\nMounts and laminates to foam boards, manually trims to size.\nEnsures timely submission of files to production.\n04/2000\n \nto \n01/2008\nWeb Designer\n \nCompany Name\n \n\u00ef\u00bc\u200b \nCity\n \n, \nState\nCreated new sites and made updates to current sites; created graphics to use on web pages; scanned documents and converted digital files\nfor links on sites; maintained updates and corrections on sites; answered email and phone call requests from departments about site changes;\nproofed pages with emails before sending live to internet.\n06/1998\n \nto \n02/2000\nDesktop Publisher\n \nCompany Name\n \n\u00ef\u00bc\u200b \nCity\n \n, \nState\nPerformed set-up and conversion of documents from Mac to PC then to UNIX systems; used QuarkXPress on Mac for the set-up of many\njobs; sent to network to be used by programmers for \"targeted\" direct mail printouts; trained new team staff members; helped with clean-up\nwhen company shut down.\n06/1997\n \nto \n03/2000\nGraphic Designer\n \nCompany Name\n \n\u00ef\u00bc\u200b \nCity\n \n, \nState\nTemporary office jobs using Macintosh computers to design files to be printed for various companies like:.\nAlltel Publishing.\nCleveland School District.\nHKM Marketing Communications.\nNationwide Advertising.\nEducation and Training\nMay 1997\nBachelor of Fine Arts\n \nAlfred University\n \n\u00ef\u00bc\u200b \nCity\n \n, \nState\nWork History\nCompany Name\nSkills\nadministrative functions, Acrobat, Adobe Dreamweaver, Photoshop, Advertising, backup, Basic, business cards, conversion, excellent customer\nservice, direct mail, email, fax, graphic design, graphics, HTML, Illustrator, Adobe InDesign, Mac, Macintosh computers, Marketing\nCommunications, Excel, mail, office, PowerPoint, Publisher, Microsoft Word, monitors, network, newsletters, takes notes, PDF, copiers, posters,\nprinter, printers, proofing, quality, QuarkXPress, supply inventory, team player, phone, UNIX, web development, web pages\n"
#         }
#     ]
# }

# # Proper data access pattern
# vectorizer = EnhancedVectorizer(max_features=100)
# all_lines = []
# for job_category in resume_data.values():
#     for resume in job_category:
#         content = resume["content"]
#         env = EnhancedResumeEnv(content)
#         line = env.reset()
#         while line:
#             all_lines.append(line)
#             line, _, _, _ = env.step("next")

# vectorizer.fit(all_lines)

# state_size = vectorizer.transform([all_lines[0]]).shape[1]
# action_size = len(ACTIONS)
# policy_net = PolicyNetwork(state_size, action_size)

# # Training loop
# for job, resumes in resume_data.items():
#     for resume in resumes:
#         env = EnhancedResumeEnv(resume["content"])
#         state = env.reset()
        
#         states, actions, rewards = [], [], []
        
#         while True:
#             if state is None:
#                 break
                
#             state_vec = vectorizer.transform([state])
#             action = policy_net.choose_action(state_vec)
#             next_state, reward, done, _ = env.step(ACTIONS[action])
            
#             states.append(state_vec)
#             actions.append(action)
#             rewards.append(reward)
            
#             if done or len(rewards) > 50:  # Early stopping
#                 break
#             state = next_state
        
#         # Compute discounted rewards
#         if rewards:
#             discounted = []
#             cumulative = 0
#             for r in reversed(rewards):
#                 cumulative = cumulative * 0.99 + r
#                 discounted.append(cumulative)
#             discounted = (discounted - np.mean(discounted)) / (np.std(discounted) + 1e-8)
            
#             # Train
#             policy_net.train(
#                 np.vstack(states),
#                 np.array(actions),
#                 discounted
#             )

# # Testing
# test_env = EnhancedResumeEnv(resume_data["DESIGNER"][0]["content"])
# state = test_env.reset()
# extracted = defaultdict(list)

# while state is not None:
#     state_vec = vectorizer.transform([state])
#     action = policy_net.choose_action(state_vec)
#     state, _, _, data = test_env.step(ACTIONS[action])
#     for k, v in data.items():
#         extracted[k] = list(set(v))  # Deduplicate

# print("Extracted Data:")
# for category, items in extracted.items():
#     print(f"{category.upper()}:")
#     for item in items:
#         print(f" - {item}")

In [7]:
# import gymnasium as gym
# from gymnasium import spaces
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv
# from collections import defaultdict
# import re
# import numpy as np

# # Load pre-trained GPT-2 model and tokenizer
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# # Add a padding token to the tokenizer
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({"pad_token": "[PAD]"})
#     model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings for the new token

# # Define the Gym-compatible environment
# class ResumeParsingEnv(gym.Env):
#     def __init__(self, resume_text):
#         super(ResumeParsingEnv, self).__init__()
#         self.lines = resume_text.split("\n")
#         self.current_line = 0
#         self.extracted_data = defaultdict(list)
        
#         # Define observation and action spaces
#         self.observation_space = spaces.Box(
#             low=0, high=tokenizer.vocab_size, shape=(100,), dtype=np.int32  # 100 tokens
#         )
#         self.action_space = spaces.Discrete(2)  # Example: 2 actions (extract or skip)

#     def reset(self, seed=None, options=None):
#         self.current_line = 0
#         self.extracted_data = defaultdict(list)
#         observation = self._get_observation()
#         return observation, {}

#     def step(self, action):
#         line = self.lines[self.current_line] if self.current_line < len(self.lines) else ""
#         reward = self._compute_reward(action, line)
    
#         self.current_line += 1
#         done = self.current_line >= len(self.lines)
        
#         next_state = self._get_observation()
    
#         info = {}  # Additional metadata (can be empty)
        
#         return next_state, reward, done, False, info  # **Now returns 5 values**


#     def _compute_reward(self, action, line):
#         if action == 1 and re.search(r"^[A-Z][a-z]+\s[A-Z][a-z]+$", line):  # Example: Extract name
#             return 1.0
#         return -1.0

#     def _get_observation(self):
#         # Ensure observation has a fixed shape (e.g., 100 elements)
#         obs = np.zeros(100)  # Default to zero if there's no data
        
#         if self.current_line < len(self.lines):
#             line_embedding = self._encode_text(self.lines[self.current_line])
            
#             # Ensure the embedding has the correct shape
#             obs[:min(len(line_embedding), 100)] = line_embedding[:100]
    
#         return obs

        
# # Initialize environment and agent
# resume_text = """PRE-PRESS GRAPHIC DESIGNER
# Summary
# Creative, hardworking designer seeking full-time position...
# Skills
# - Adobe InDesign
# - Photoshop
# Experience
# 01/2008 to Current: Graphic Designer at Company
# """

# # Wrap the environment for PPO compatibility
# env = DummyVecEnv([lambda: ResumeParsingEnv(resume_text)])
# rl_agent = PPO("MlpPolicy", env, verbose=1)

# # Train the agent
# rl_agent.learn(total_timesteps=2000)

# # Test the agent
# test_env = ResumeParsingEnv(resume_text)
# observation, _ = test_env.reset()
# while True:
#     action, _ = rl_agent.predict(observation)
#     observation, reward, done, truncated, info = test_env.step(action)
#     if done:
#         break
# print("Extracted Data:", test_env.extracted_data)

In [8]:
from datasets import load_dataset

# Replace with the dataset name or path
dataset_name = "ganchengguang/resume_seven_class"  # Example: "imdb", "glue", etc.

# Load the dataset (some datasets may require specifying a split)
dataset = load_dataset(dataset_name)

# Print dataset information
print(dataset)

# Display the first few rows of the dataset
print(dataset['train'][2])  # Change 'train' to the relevant split (e.g., 'test', 'validation')
data = dataset


README.md:   0%|          | 0.00/528 [00:00<?, ?B/s]

resume.txt:   0%|          | 0.00/5.02M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78670 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 78670
    })
})
{'text': 'PI\tPhone: 940-242-3303'}


In [9]:
import csv

# Read the dataset from a text file
input_file = "/kaggle/input/resume/resume.txt"  # Change this to the actual filename
output_file = "processed_data.csv"

data = []

# Open the text file and process each line
with open(input_file, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue  # Skip empty lines

        # Split at the first tab character
        parts = line.split("\t", 1)
        if len(parts) == 2:
            label, text = parts
            data.append([label, text])

# Save to a CSV file
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Label", "Text"])  # Write header
    writer.writerows(data)

print(f"Processed data saved to {output_file}")


Processed data saved to processed_data.csv


In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the model and tokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load dataset from CSV
dataset = load_dataset("csv", data_files="/kaggle/working/processed_data.csv")

# Let's first examine the dataset structure
print("Dataset structure:", dataset)
print("Column names:", dataset["train"].column_names)
print("Sample row:", dataset["train"][0])

# Create a label mapping dictionary
# Assuming your Label column contains categorical values like "Exp", "Loc", etc.
# First, get unique labels
unique_labels = set()
for example in dataset["train"]:
    unique_labels.add(example["Label"])
unique_labels = sorted(list(unique_labels))
print(f"Unique labels found: {unique_labels}")

# Create mapping dictionary
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for i, label in enumerate(unique_labels)}
print(f"Label mapping: {label_to_id}")

# Now update the number of labels
num_labels = len(unique_labels)
print(f"Number of labels: {num_labels}")

# Split dataset into train and validation sets
dataset = dataset['train'].train_test_split(test_size=0.1)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['Label', 'Text'],
        num_rows: 78669
    })
})
Column names: ['Label', 'Text']
Sample row: {'Label': 'Exp', 'Text': 'Name: Abiral Pandey'}
Unique labels found: ['Edu', 'Exp', 'Obj', 'PI', 'QC', 'Skill', 'Sum']
Label mapping: {'Edu': 0, 'Exp': 1, 'Obj': 2, 'PI': 3, 'QC': 4, 'Skill': 5, 'Sum': 6}
Number of labels: 7


In [11]:

def tokenize_and_prepare(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(
        examples["Text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    
    # Convert categorical labels to numeric labels using our mapping
    tokenized_inputs["labels"] = [label_to_id[label] for label in examples["Label"]]
    
    return tokenized_inputs

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_and_prepare, batched=True)

# Define compute_metrics function for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



Map:   0%|          | 0/70802 [00:00<?, ? examples/s]

Map:   0%|          | 0/7867 [00:00<?, ? examples/s]

In [12]:
# Define model - using SequenceClassification instead of TokenClassification
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

# Move model to GPU if available
model.to(device)

# Training arguments with progress display
training_args = TrainingArguments(
    output_dir="./classification_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Custom callback to print progress during training
from transformers import TrainerCallback

class EpochProgressCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        if state.is_local_process_zero:
            print(f"\n{'='*50}")
            print(f"End of epoch {state.epoch}")
            print(f"Steps completed: {state.global_step}")
            print(f"{'='*50}\n")
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs:
            loss = logs.get('loss', None)
            lr = logs.get('learning_rate', None)
            step = state.global_step
            
            if loss is not None:
                print(f"Step {step}: Loss: {loss:.4f}", end=" ")
            if lr is not None:
                print(f"LR: {lr:.2e}", end=" ")
            if 'eval_loss' in logs:
                print(f"\nEvaluation metrics: {logs}")
            else:
                print()



In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EpochProgressCallback()]
)

# Train model
print("Starting training...")
trainer.train()

# Save the model
print("Saving the model...")
trainer.save_model("./classification_model_final")
print("Training complete!")


<ipython-input-14-f177cf927f4d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.571500,0.450999,0.862082,0.858335,0.858262,0.862082
2,0.272000,0.446739,0.868056,0.867570,0.867355,0.868056


Step 10: Loss: 1.7597 LR: 2.00e-05 
Step 20: Loss: 1.4818 LR: 2.00e-05 
Step 30: Loss: 1.3016 LR: 2.00e-05 
Step 40: Loss: 1.2815 LR: 1.99e-05 
Step 50: Loss: 1.1281 LR: 1.99e-05 
Step 60: Loss: 1.0653 LR: 1.99e-05 
Step 70: Loss: 1.1103 LR: 1.99e-05 
Step 80: Loss: 0.9353 LR: 1.99e-05 
Step 90: Loss: 0.9740 LR: 1.99e-05 
Step 100: Loss: 0.9767 LR: 1.98e-05 
Step 110: Loss: 0.8084 LR: 1.98e-05 
Step 120: Loss: 0.8760 LR: 1.98e-05 
Step 130: Loss: 0.8447 LR: 1.98e-05 
Step 140: Loss: 0.7957 LR: 1.98e-05 
Step 150: Loss: 0.8862 LR: 1.98e-05 
Step 160: Loss: 0.7822 LR: 1.98e-05 
Step 170: Loss: 0.7690 LR: 1.97e-05 
Step 180: Loss: 0.6229 LR: 1.97e-05 
Step 190: Loss: 0.8209 LR: 1.97e-05 
Step 200: Loss: 0.8482 LR: 1.97e-05 
Step 210: Loss: 0.7628 LR: 1.97e-05 
Step 220: Loss: 0.6699 LR: 1.97e-05 
Step 230: Loss: 0.6547 LR: 1.97e-05 
Step 240: Loss: 0.7764 LR: 1.96e-05 
Step 250: Loss: 0.7434 LR: 1.96e-05 
Step 260: Loss: 0.6963 LR: 1.96e-05 
Step 270: Loss: 0.9167 LR: 1.96e-05 
Step 280: 

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step 4430: Loss: 0.3571 LR: 1.33e-05 
Step 4440: Loss: 0.4932 LR: 1.33e-05 
Step 4450: Loss: 0.4606 LR: 1.33e-05 
Step 4460: Loss: 0.4001 LR: 1.33e-05 
Step 4470: Loss: 0.3518 LR: 1.33e-05 
Step 4480: Loss: 0.3432 LR: 1.33e-05 
Step 4490: Loss: 0.4420 LR: 1.32e-05 
Step 4500: Loss: 0.4406 LR: 1.32e-05 
Step 4510: Loss: 0.5070 LR: 1.32e-05 
Step 4520: Loss: 0.3711 LR: 1.32e-05 
Step 4530: Loss: 0.4716 LR: 1.32e-05 
Step 4540: Loss: 0.2981 LR: 1.32e-05 
Step 4550: Loss: 0.3158 LR: 1.31e-05 
Step 4560: Loss: 0.3180 LR: 1.31e-05 
Step 4570: Loss: 0.4814 LR: 1.31e-05 
Step 4580: Loss: 0.4412 LR: 1.31e-05 
Step 4590: Loss: 0.3783 LR: 1.31e-05 
Step 4600: Loss: 0.3767 LR: 1.31e-05 
Step 4610: Loss: 0.3995 LR: 1.31e-05 
Step 4620: Loss: 0.3194 LR: 1.30e-05 
Step 4630: Loss: 0.3817 LR: 1.30e-05 
Step 4640: Loss: 0.4399 LR: 1.30e-05 
Step 4650: Loss: 0.4004 LR: 1.30e-05 
Step 4660: Loss: 0.4237 LR: 1.30e-05 
Step 4670: Loss: 0.3945 LR: 1.30e-05 
Step 4680: Loss: 0.3528 LR: 1.30e-05 
Step 4690: L

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step 8860: Loss: 0.2411 LR: 6.65e-06 
Step 8870: Loss: 0.3847 LR: 6.64e-06 
Step 8880: Loss: 0.3806 LR: 6.62e-06 
Step 8890: Loss: 0.3777 LR: 6.61e-06 
Step 8900: Loss: 0.3444 LR: 6.59e-06 
Step 8910: Loss: 0.2954 LR: 6.58e-06 
Step 8920: Loss: 0.2506 LR: 6.56e-06 
Step 8930: Loss: 0.4206 LR: 6.55e-06 
Step 8940: Loss: 0.3081 LR: 6.53e-06 
Step 8950: Loss: 0.2221 LR: 6.52e-06 
Step 8960: Loss: 0.3777 LR: 6.50e-06 
Step 8970: Loss: 0.3609 LR: 6.49e-06 
Step 8980: Loss: 0.2302 LR: 6.47e-06 
Step 8990: Loss: 0.3755 LR: 6.46e-06 
Step 9000: Loss: 0.2881 LR: 6.44e-06 
Step 9010: Loss: 0.3152 LR: 6.43e-06 
Step 9020: Loss: 0.3093 LR: 6.41e-06 
Step 9030: Loss: 0.3434 LR: 6.40e-06 
Step 9040: Loss: 0.3238 LR: 6.38e-06 
Step 9050: Loss: 0.3580 LR: 6.37e-06 
Step 9060: Loss: 0.2608 LR: 6.35e-06 
Step 9070: Loss: 0.3329 LR: 6.34e-06 
Step 9080: Loss: 0.2525 LR: 6.32e-06 
Step 9090: Loss: 0.2171 LR: 6.31e-06 
Step 9100: Loss: 0.5496 LR: 6.29e-06 
Step 9110: Loss: 0.4484 LR: 6.28e-06 
Step 9120: L

# Model Testing

Process a pdf

In [12]:
import PyPDF2

def pdf_to_text(pdf_path, txt_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Initialize an empty string to hold the text
        text = ''
        
        # Loop through each page in the PDF
        for page_num in range(len(pdf_reader.pages)):
            # Get the page object
            page = pdf_reader.pages[page_num]
            
            # Extract the text from the page
            text += page.extract_text()
        
        # Write the extracted text to a .txt file
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

# Example usage
pdf_path = '/kaggle/input/resume-dataset/data/data/ENGINEERING/10030015.pdf'  # Path to the input PDF file
txt_path = 'output.txt'   # Path to the output text file

pdf_to_text(pdf_path, txt_path)
print(f"Text extracted from {pdf_path} and saved to {txt_path}")

Text extracted from /kaggle/input/resume-dataset/data/data/ENGINEERING/10030015.pdf and saved to output.txt


# Code for prediction

In [13]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
# import torch
# import numpy as np

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Define the path to the fine-tuned model checkpoint
# model_checkpoint = "/kaggle/input/bert_fintuned_resume/pytorch/v1/1"

# # Load the fine-tuned model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# # Move the model to the appropriate device (GPU or CPU)
# model.to(device)

# # Function to preprocess and tokenize input text
# def preprocess_and_tokenize(texts):
#     return tokenizer(
#         texts,
#         padding="max_length",
#         truncation=True,
#         max_length=512,
#         return_tensors="pt"
#     )

# # Function to make predictions
# def predict(texts):
#     # Preprocess and tokenize the input texts
#     inputs = preprocess_and_tokenize(texts)
    
#     # Move inputs to the same device as the model
#     inputs = {key: value.to(device) for key, value in inputs.items()}
    
#     # Perform inference
#     with torch.no_grad():
#         outputs = model(**inputs)
    
#     # Get predicted class probabilities
#     logits = outputs.logits
#     probabilities = torch.softmax(logits, dim=-1)
    
#     # Get predicted class labels
#     predictions = torch.argmax(probabilities, dim=-1).cpu().numpy()
    
#     return predictions, probabilities.cpu().numpy()

# # Example input texts for prediction
# input_texts = [
#     "This is an example text for prediction.",
#     "Another example text to classify.",
#     "This is a third example."
# ]

# # Make predictions
# predictions, probabilities = predict(input_texts)

# # Print results
# for i, (text, pred, prob) in enumerate(zip(input_texts, predictions, probabilities)):
#     print(f"Text {i + 1}: {text}")
#     print(f"Predicted Label: {pred}")
#     print(f"Class Probabilities: {prob}")
#     print("-" * 50)

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the path to the fine-tuned model checkpoint
model_checkpoint = "/kaggle/input/bert_fintuned_resume/pytorch/v1/1/checkpoint"

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Move the model to the appropriate device (GPU or CPU)
model.to(device)

# Define the label mapping
id_to_label = {
    0: "Education",
    1: "Experience",
    2: "Objectives",
    3: "Personal Information",
    4: "QC",
    5: "Skill",
    6: "Summary"
}

# Function to preprocess and tokenize input text
def preprocess_and_tokenize(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Function to make predictions
def predict(texts):
    # Preprocess and tokenize the input texts
    inputs = preprocess_and_tokenize(texts)
    
    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get predicted class probabilities
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    
    # Get predicted class labels
    predictions = torch.argmax(probabilities, dim=-1).cpu().numpy()
    
    return predictions, probabilities.cpu().numpy()

# Load text data from a .txt file
def load_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = file.readlines()
    # Remove newline characters and strip whitespace
    texts = [text.strip() for text in texts]
    return texts

# Path to the .txt file
txt_file_path = "/kaggle/working/output.txt"

# Load and preprocess the text data
texts = load_text_data(txt_file_path)

# Make predictions
predictions, probabilities = predict(texts)

# Decode predictions using id_to_label mapping
decoded_predictions = [id_to_label[pred] for pred in predictions]

# Print results
for i, (text, pred, decoded_pred, prob) in enumerate(zip(texts, predictions, decoded_predictions, probabilities)):
    print(f"Text {i + 1}: {text}")
    print(f"Predicted Label ID: {pred}")
    print(f"Decoded Label: {decoded_pred}")
    print(f"Class Probabilities: {prob}")
    print("-" * 50)

Using device: cpu
Text 1: ENGINEERING LAB TECHNICIAN
Predicted Label ID: 1
Decoded Label: Experience
Class Probabilities: [0.06409964 0.6951933  0.00362999 0.09245677 0.01657899 0.09101989
 0.03702145]
--------------------------------------------------
Text 2: Career Focus
Predicted Label ID: 2
Decoded Label: Objectives
Class Probabilities: [0.01421985 0.15456252 0.35557592 0.1947772  0.00694555 0.03790122
 0.23601773]
--------------------------------------------------
Text 3: My main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my
Predicted Label ID: 2
Decoded Label: Objectives
Class Probabilities: [0.00118166 0.0143166  0.90888274 0.02801012 0.00164228 0.00335065
 0.04261587]
--------------------------------------------------
Text 4: skills and continue to gain experience in the aerospace industry to advance in my career.
Predicted Label ID: 2
Decoded Label: Objectives
Class Probabilities: [0.0014273  

Sample row: {'Label': 'Exp', 'Text': 'Name: Abiral Pandey'}
Unique labels found: ['Edu', 'Exp', 'Obj', 'PI', 'QC', 'Skill', 'Sum']
Label mapping: {'Edu': 0, 'Exp': 1, 'Obj': 2, 'PI': 3, 'QC': 4, 'Skill': 5, 'Sum': 6}
Number of labels: 7
